In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

import random
import os

In [2]:
SEED = 67

## Vista general del conjunto de imágenes

In [3]:
INPUT_PATH = "/kaggle/input/butterfly-image-classification/"

In [4]:
df_train = pd.read_csv(f"{INPUT_PATH}Training_set.csv")
df_test = pd.read_csv(f"{INPUT_PATH}Testing_set.csv")

In [5]:
df_train.sample(5, random_state=SEED)

,filename,label
369,Image_370.jpg,TWO BARRED FLASHER
534,Image_535.jpg,AN 88
2356,Image_2357.jpg,MILBERTS TORTOISESHELL
999,Image_1000.jpg,SLEEPY ORANGE
4777,Image_4778.jpg,ORANGE TIP


In [6]:
df_test.sample(5, random_state=SEED)

,filename
75,Image_76.jpg
1069,Image_1070.jpg
2091,Image_2092.jpg
1463,Image_1464.jpg
1822,Image_1823.jpg


In [7]:
print(f"Hay {df_train.shape[0]} en el conjunto de entrenamiento")
print(f"Hay {df_test.shape[0]} en el conjunto de test")

Hay 6499 en el conjunto de entrenamiento
Hay 2786 en el conjunto de test


In [8]:
subfolders = [f.path for f in os.scandir(INPUT_PATH) if f.is_dir()]
for s in subfolders:
    print(f"Hay {len(os.listdir(s))} en el conjunto de {os.path.basename(s)}")

Hay 2786 en el conjunto de test
Hay 6499 en el conjunto de train


El número de imágenes en los registros y en las carpetas coinciden, por tanto carga de datos correcta

In [9]:
labels = df_train['label'].unique()
print(f"Hay {len(labels)} tipos de mariposa")
print(f"Algunas de ellas {labels[:5]}")

Hay 75 tipos de mariposa
Algunas de ellas ['SOUTHERN DOGFACE' 'ADONIS' 'BROWN SIPROETA' 'MONARCH'
 'GREEN CELLED CATTLEHEART']


In [10]:
fig = px.histogram(df_train, x='label')
fig.show()

In [11]:
df_train['label'].value_counts()

label
MOURNING CLOAK    131
SLEEPY ORANGE     107
ATALA             100
BROWN SIPROETA     99
CRECENT            97
                 ... 
AMERICAN SNOOT     74
GOLD BANDED        73
MALACHITE          73
CRIMSON PATCH      72
WOOD SATYR         71
Name: count, Length: 75, dtype: int64

- Las distribuciones de las clases es bastante uniforme, un rango de entre 70 y 100 fotos.
- Destaca una etiqueta `MOURNING CLOAK` con unas 130 fotos.

**ESTRATEGIA 1** Utilizar técnicas de aumento de datos para equiparar las etiquetas. Sería un aumento de datos previo al entrenamiento de los modelos de redes neuronales y no en el vuelo (que es otra técnica más)


**ESTRATEGIA 2** Reducir el número de clases, centrándonos en las que más nos interesen.

Para poder trabajar con las imágenes, es esencial seguir una estructura de directorios para que pueda ser cargados por `tensorflow.keras`
Los pasos a seguir son los siguientes:

1. Partir el conjunto de entrenamiento para tener un conjunto de validación
2. Crear la estructura de carpetas por clases
3. Cargar el conjunto de datos con la función `image_dataset_from_directory`